In [2]:
import logging
import pandas as pd
import os
import shutil
import time

PARQUET_FILE_PATH = 'parquet/q-repo-list_q-kdb.parquet'

GITHUB_REPOS_DIR = 'github-repos'
TARGET_DIR = 'training-set-q-language'

SLEEP_SECONDS = 5

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

df = pd.read_parquet(PARQUET_FILE_PATH)
sorted_df = df.sort_values('stargazers_count', ascending=False).reset_index(drop=True)


DESIRED_EXTENSIONS = ('.q', '.md')

def copy_desired_files(org_name, repo_name):
    """
    Copies files with desired extensions from a specified GitHub repository to a target directory.

    This function traverses the directory structure of a given repository, filters out hidden files
    and directories, and copies files with specified extensions to a target directory while preserving
    the directory structure.

    Args:
        org_name (str): The name of the GitHub organization.
        repo_name (str): The name of the GitHub repository.

    Returns:
        None
    """
    repo_path = os.path.join(GITHUB_REPOS_DIR, org_name, repo_name)
    for root, dirs, files in os.walk(repo_path):
        # `root` is the current directory
        # `dirs` holds the list of directories in the current directory
        # `files` holds the list of files in the current directory

        # Remove hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        # Remove hidden files
        files = [f for f in files if not f.startswith('.')]

        rel_path = os.path.relpath(root, GITHUB_REPOS_DIR)
        os.makedirs(os.path.join(TARGET_DIR, rel_path), exist_ok=True)

        for file in files:
            if file.endswith(DESIRED_EXTENSIONS):
                src_file = os.path.join(root, file)
                dest_file = os.path.join(TARGET_DIR, rel_path, file)
                shutil.copy(src_file, dest_file)


# Loop through the DataFrame and download the repositories
for index, row in sorted_df.iterrows():
    # Keep org_name and repo_name separate since path separator may be different on different OS.
    org_name = row['owner']['login']
    repo_name = row['name']
    logging.info(f"Processing repository {index} : {org_name}/{repo_name}...")
    copy_desired_files(org_name, repo_name)


logging.info('Done')

2024-11-08 09:36:59,902 - INFO - Processing repository 0 : KxSystems/kdb...
2024-11-08 09:37:04,041 - INFO - Processing repository 1 : KxSystems/pyq...
2024-11-08 09:37:05,039 - INFO - Processing repository 2 : psaris/qtips...
2024-11-08 09:37:06,151 - INFO - Processing repository 3 : psaris/funq...
2024-11-08 09:37:11,167 - INFO - Processing repository 4 : qbists/studyq...
2024-11-08 09:37:19,131 - INFO - Processing repository 5 : KxSystems/jupyterq...
2024-11-08 09:37:20,772 - INFO - Processing repository 6 : KxSystems/embedPy...
2024-11-08 09:37:21,736 - INFO - Processing repository 7 : KxSystems/docs...
2024-11-08 09:37:57,744 - INFO - Processing repository 8 : KxSystems/ml...
2024-11-08 09:38:16,652 - INFO - Processing repository 9 : KxSystems/javakdb...
2024-11-08 09:38:17,948 - INFO - Processing repository 10 : KxSystems/kafka...
2024-11-08 09:38:18,835 - INFO - Processing repository 11 : KxSystems/rkdb...
2024-11-08 09:38:19,269 - INFO - Processing repository 12 : jlas/ml.q...
